<a href="https://colab.research.google.com/github/Lucas-Monteiro-Henriques/Implement_VISIONTS/blob/main/MAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importei MAE do Facebook

In [1]:
import numpy as np
import pandas as pd
import sys
import os
import subprocess
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks
from skimage.transform import resize
import urllib.request


import torch
import timm
from timm.models.vision_transformer import PatchEmbed, Block
np.float = float

import matplotlib.pyplot as plt
from PIL import Image

In [2]:
repo_dir = 'mae'
repo_url = 'https://github.com/facebookresearch/mae.git'

if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.4.5  # 0.3.2 does not work in Colab
    if not os.path.exists('mae'):
        !git clone https://github.com/facebookresearch/mae.git
    sys.path.append('./mae')
else:
    if not os.path.exists(repo_dir):
        print(f'Cloned in : {repo_dir}')
        subprocess.run(['git', 'clone', repo_url], check=True)
    else:
        print(f'Repository already exists in {repo_dir}')

Running in Colab.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.

Cloning into 'mae'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39 (from 1)
Receiving objects: 100% (39/39), 832.32 KiB | 9.68 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [3]:
if os.path.exists('pytorch_image_models'):
    pass
else:
    !git clone https://github.com/huggingface/pytorch-image-models.git
    os.rename('pytorch-image-models', 'pytorch_image_models') #renomeia a pasta porque dá conflito nos imports


Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 20924, done.
remote: Counting objects: 100% (371/371), done.
remote: Compressing objects: 100% (159/159), done.
remote: Total 20924 (delta 280), reused 213 (delta 212), pack-reused 20553 (from 3)
Receiving objects: 100% (20924/20924), 28.04 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (15531/15531), done.


In [4]:
# from pytorch_image_models.timm.models.vision_transformer import PatchEmbed, Block

In [5]:
mae_path = os.path.abspath("mae")
if mae_path not in sys.path:
    sys.path.insert(0, mae_path)

from models_mae import mae_vit_base_patch16

In [6]:
model = mae_vit_base_patch16()

TypeError: Block.__init__() got an unexpected keyword argument 'qk_scale'

Implementação do Algoritmo de transformação das séries temporais 1D para 2D

In [ ]:
df = pd.read_csv("https://github.com/zhouhaoyi/ETDataset/raw/main/ETT-small/ETTm1.csv")


serie = df["OT"].values[:512]

acf_vals = acf(serie, nlags=min(50, len(serie)//2), fft=True)

lags = np.arange(1, len(acf_vals))
acf_sem_lag0 = acf_vals[1:]
peaks, _ = find_peaks(acf_sem_lag0, height=0.2)

if len(peaks) == 0:
    P = 1
else:
    P = peaks[0] + 1

print(f"Periodicidade estimada: {P}")

L = (len(serie) // P) * P
serie_cortada = serie[:L]


matriz_2d = serie_cortada.reshape(P, L // P)
print("Matriz 2D:\n", matriz_2d)

Normalization do Iraw


In [ ]:
import numpy as np
from statistics import mean
desvio_padrao = np.std(matriz_2d)
print(desvio_padrao)
mean  = np.mean(matriz_2d)
print(mean)

In [ ]:
r = 0.4
Inorm = r * (matriz_2d - mean)/ desvio_padrao
print(Inorm)

Esse é o algorimo do Alignment, mas não foi testado, pois o MAE do Facebook precisa ser alterado na parte da mascara


In [ ]:
L = 300
H = 100
c = 0.4
S = 1
N = max(Inorm.shape)

# Etapa 1: Render Igray (imagem 3 canais iguais)
Igray = np.stack([Inorm] * 3, axis=-1)

# Etapa 2: Alignment
n = int(np.floor(c * N * L / (L + H)))  # número de patches visíveis na horizontal
target_shape = (N * S, n * S)           # shape final da imagem após resize

# Resize Igray (mantendo apenas 1 canal para simplificar o MAE)
Igray_resized = resize(Inorm, target_shape, order=1, mode='reflect', anti_aliasing=False)

# Converter para imagem 3 canais (opcional, dependendo do input do MAE)
Igray_resized_3ch = np.stack([Igray_resized] * 3, axis=-1)

# Visualização
plt.imshow(Igray_resized, cmap='gray', aspect='auto')
plt.title("Igray redimensionado para Alignment (VisionTS)")
plt.colorbar()
plt.show()

# Print das formas para verificação
print("Shape original Inorm:", Inorm.shape)
print("Shape Igray:", Igray.shape)
print("Shape Igray_resized:", Igray_resized.shape)
print("Shape Igray_resized_3ch:", Igray_resized_3ch.shape)


Funções do MAE

In [ ]:
# define the utils

imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std = np.array([0.229, 0.224, 0.225])

def show_image(image, title=''):
    # image is [H, W, 3]
    assert image.shape[2] == 3
    plt.imshow(torch.clip((image * imagenet_std + imagenet_mean) * 255, 0, 255).int())
    plt.title(title, fontsize=16)
    plt.axis('off')
    return

def prepare_model(chkpt_dir, arch='mae_vit_large_patch16'):
    # build model
    from models_mae import mae_vit_base_patch16
    model = mae_vit_base_patch16()

    # load model
    checkpoint = torch.load(chkpt_dir, map_location='cpu')
    msg = model.load_state_dict(checkpoint['model'], strict=False)
    print(msg)
    return model

def run_one_image_from_aligned_image(Igray_resized_3ch, model, L=300, H=100, c=0.4):
    """
    Igray_resized_3ch: imagem com 3 canais (já alinhada), shape (H, W, 3), valores já normalizados
    model: modelo MAE do repo da Meta (Vit-base etc)
    L, H, c: hiperparâmetros do VisionTS
    """
    # 1. Redimensionar para 224x224 (como esperado pelo MAE)
    img = resize(Igray_resized_3ch, (224, 224), order=1, mode='reflect', anti_aliasing=False)
    x = torch.tensor(img, dtype=torch.float32).unsqueeze(0)       # [1, H, W, C]
    if False:
        x = torch.einsum('nhwc->nchw', x).cuda()                      # [1, 3, 224, 224]
    if True:
        x = torch.einsum('nhwc->nchw', x).to(device)

    # 2. Criar máscara estruturada: lado direito mascarado
    patch_size = model.patch_embed.patch_size[0]         # geralmente 16
    num_patches_row = x.shape[2] // patch_size           # 224 / 16 = 14
    num_patches = num_patches_row ** 2                   # 14 x 14 = 196

    N = num_patches_row
    n = int(np.floor(c * N * L / (L + H)))                # número de colunas visíveis

    # Criar máscara: 0 = visível (esquerda), 1 = mascarado (direita)
    mask = torch.ones(num_patches, device=x.device)
    for row in range(N):
        for col in range(n):  # colunas visíveis
            idx = row * N + col
            mask[idx] = 0
    mask = mask.unsqueeze(0)  # [1, num_patches]

    # 3. Forward com máscara estruturada
    with torch.no_grad():
        if False:
            loss, y, _ = model(x.float(), mask=mask)         # usamos mask diretamente
        if True:
            loss, y, _ = model(x.float())
        y = model.unpatchify(y)                          # [1, 3, H, W]
        if False:
            y = torch.einsum('nchw->nhwc', y).cpu().numpy()
        if True:
            y = torch.einsum('nchw->nhwc', y).cpu()

        mask_recon = mask.unsqueeze(-1).repeat(1, 1, patch_size ** 2 * 3)
        mask_recon = model.unpatchify(mask_recon)
        mask_recon = torch.einsum('nchw->nhwc', mask_recon).cpu()

    x_vis = x.cpu()
    x_vis = torch.einsum('nchw->nhwc', x_vis)

    im_masked = x_vis * (1 - mask_recon)
    im_paste = x_vis * (1 - mask_recon) + y * mask_recon

    # 4. Visualização
    plt.rcParams['figure.figsize'] = [24, 6]
    plt.subplot(1, 4, 1); show_image(x_vis[0], "original")
    plt.subplot(1, 4, 2); show_image(im_masked[0], "structured mask")
    plt.subplot(1, 4, 3); show_image(y[0], "reconstruction")
    plt.subplot(1, 4, 4); show_image(im_paste[0], "reconstruction + visible")
    plt.show()

    return y[0]  # retorno da imagem reconstruída (H, W, 3)


Vizulização da imagem

In [ ]:
# Suponha que você já tem:
# Igray_resized_3ch com shape (29, 8, 3) — imagem com 3 canais e normalizada (VisionTS-style)

# Etapa 1: Redimensionar para 224 x 224 (como esperado pelo MAE)
img = resize(Igray_resized_3ch, (224, 224), order=1, mode='reflect', anti_aliasing=False)

# Etapa 2: Visualização opcional (sem normalizar com ImageNet)
plt.rcParams['figure.figsize'] = [5, 5]
plt.imshow(img)
plt.title("Input aligned (VisionTS)")
plt.axis('off')
plt.show()

# Se for usar a função run_one_image, ela espera um tensor com valores normalizados já
# Portanto você pode usar:
img = torch.tensor(img, dtype=torch.float32)


Download dos pesos de Modelo


In [ ]:
# MAE com decoder
if False:
    !wget -nc https://dl.fbaipublicfiles.com/mae/visualize/mae_visualize_vit_base.pth

if True:
    url = 'https://dl.fbaipublicfiles.com/mae/visualize/mae_visualize_vit_base.pth'
    dest = 'mae_visualize_vit_base.pth'

    if not os.path.exists(dest):
        print('Downloading MAE with decoder...')
        urllib.request.urlretrieve(url, dest)
    else:
        pass


In [ ]:
# Carrega o modelo base

from models_mae import mae_vit_base_patch16

chkpt_dir = 'mae_visualize_vit_base.pth'
model = mae_vit_base_patch16()
checkpoint = torch.load(chkpt_dir, map_location='cpu')
msg = model.load_state_dict(checkpoint['model'], strict=False)
print(msg)

# Enviar para GPU
model.eval()
# model.cuda()

In [ ]:
# make random mask reproducible (comment out to make it change)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # usa cuda ou cpu
model = model.to(device)

# Garante reprodutibilidade na máscara aleatória
torch.manual_seed(2)

Igray_resized_3ch_tensor = torch.from_numpy(Igray_resized_3ch).float().to(device)
print('MAE with pixel reconstruction:')
reconstructed_image = run_one_image_from_aligned_image(Igray_resized_3ch, model)


Algoritmo de reconstrução, mudar o L e H  em def reconstruct_and_evaluate_forecast para
Tabela oficial dos resultados (extraída do artigo)


In [ ]:
# 📊 Tabela oficial dos resultados no dataset ETTm1:
# -----------------------------------------------------------
# | L   | H   | MSE ↓   | MAE ↓   |
# |-----|-----|---------|---------|
# | 96  | 96  | 0.234   | 0.368   |
# | 96  | 192 | 0.286   | 0.411   |
# | 96  | 336 | 0.287   | 0.414   |
# | 96  | 720 | 0.365   | 0.470   |
# -----------------------------------------------------------
#

In [ ]:
def reconstruct_and_evaluate_forecast(y_reconstructed, original_Inorm):
    """
    y_reconstructed: imagem reconstruída pelo MAE, shape (H, W, 3)
    original_Inorm: matriz 2D usada originalmente, shape (P, L/P)
    r: fator de normalização aplicado
    """
    r = 0.4
    import numpy as np
    from skimage.transform import resize
    from sklearn.metrics import mean_squared_error, mean_absolute_error

    # Etapa 1: média dos 3 canais para obter imagem grayscale reconstruída
    recon_gray = y_reconstructed.mean(axis=-1)  # shape (H, W)

    # Etapa 2: resize reverso para shape original
    recon_resized_back = resize(recon_gray, original_Inorm.shape, order=1, mode='reflect', anti_aliasing=False)

    # Etapa 3: flatten
    recon_flat = recon_resized_back.flatten()
    orig_flat = original_Inorm.flatten()

    # Etapa 4: desnormalizar
    mean_orig = np.mean(orig_flat / r)
    std_orig = np.std(orig_flat / r)
    recon_deno = recon_flat / r
    recon_deno = recon_deno * std_orig + mean_orig

    # Etapa 5: extrair a janela de previsão
    P, total_steps = original_Inorm.shape
    H = 96
    L = 96
    pred_steps = int(0.4 * L * total_steps / (L + H))
    forecast_start = total_steps - pred_steps

    recon_forecast = recon_deno.reshape(P, total_steps)[:, forecast_start:]
    target_forecast = (original_Inorm / r).reshape(P, total_steps)[:, forecast_start:]
    target_forecast = target_forecast * std_orig + mean_orig

    mse = mean_squared_error(target_forecast.flatten(), recon_forecast.flatten())
    mae = mean_absolute_error(target_forecast.flatten(), recon_forecast.flatten())

    print(f"MSE na janela de previsão: {mse:.6f}")
    print(f"MAE na janela de previsão: {mae:.6f}")

    return recon_forecast, target_forecast, mse


In [ ]:

recon_forecast, target_forecast, mse = reconstruct_and_evaluate_forecast(
    y_reconstructed=reconstructed_image.numpy(),
    original_Inorm=Inorm
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
plt.plot(target_forecast.flatten(), label='target', color='gray')
plt.plot(recon_forecast.flatten(), label='prediction', color='steelblue')
plt.title(f"Forecast (últimos {target_forecast.shape[1]} steps) - MSE: {mse:.4f}")
plt.xlabel("Tempo")
plt.ylabel("Valor")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()
